In [ ]:
#!pip install nltk

In [1]:
import numpy as np
import pandas as pd
from SportsExperiencePlatform.data import connect_db, get_data, upload_file_to_gs
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import save_npz, load_npz
import re
from langdetect import detect
from googletrans import Translator
import random
import datetime
from datetime import date
import argostranslate.package, argostranslate.translate
import json

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [2]:
conn = connect_db()

if conn:
    users_df, events_df, ahoy_events_df = get_data(conn)

/home/daniel/.pyenv/versions/3.8.12/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/daniel/.pyenv/versions/3.8.12/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/daniel/.pyenv/versions/3.8.12/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [75]:
ahoy_events_df

,id,visit_id,user_id,name,properties,time
0,72,149,66,View Offer Page ID 4274,"{'user': 66, 'offer': 4274}",2022-06-07 19:05:10.058669
1,73,149,66,View Offer Page ID 4090,"{'user': 66, 'offer': 4090}",2022-06-07 19:06:29.831288
2,74,149,66,View Offer Page ID 4662,"{'user': 66, 'offer': 4662}",2022-06-07 19:12:24.105959
3,75,149,66,View Offer Page ID 4090,"{'user': 66, 'offer': 4090}",2022-06-07 19:13:07.936900
4,76,149,66,View Offer Page ID 4090,"{'user': 66, 'offer': 4090}",2022-06-07 19:16:52.860923
5,77,149,66,View Offer Page ID 4090,"{'user': 66, 'offer': 4090}",2022-06-07 19:18:46.248524
6,78,149,66,View Offer Page ID 4665,"{'user': 66, 'offer': 4665}",2022-06-07 19:22:01.135433
7,79,149,66,View Offer Page ID 4088,"{'user': 66, 'offer': 4088}",2022-06-07 19:22:09.810442
8,80,149,66,View Offer Page ID 4090,"{'user': 66, 'offer': 4090}",2022-06-07 19:22:21.022513
9,81,149,66,View Offer Page ID 4087,"{'user': 66, 'offer': 4087}",2022-06-07 19:27:21.890835


In [74]:
users_df[users_df['id'] == 66]

,id,email,encrypted_password,reset_password_token,reset_password_sent_at,remember_created_at,created_at,updated_at,first_name,last_name,description,date_of_birth,gender,location,latitude,longitude,preferences
8,66,johanna@gmail.com,$2a$12$asPE07JW633q4rpV2g52j.7ea68mf3/PGzAX0Vx...,None,None,None,2022-06-07 18:16:15.070114,2022-06-07 19:41:46.527553,Maria,Maier,"Hi, my name is Johanna! I want to go on my nex...",1993-01-25,female,Berlin,52.520008,13.404954,"[197, 198, 199, 200, 201, 202]"


In [40]:
events_df[events_df['id'].isin([4274, 4090, 4662])]

,id,title,description,price_per_person,capacity,address,latitude,longitude,is_external,url,offer_date,offer_time,created_at,updated_at,user_id,tag_id,external_image_url
3,4090,Kajak Berlin Tours,Enjoy the stunning waterscape of the legendary...,25,15.0,"Kayak-Truck vis-á-vis Cafe A.Horn, Carl-Herz-U...",52.495747,13.405571,False,None,2022-06-09,21:14:41,2022-06-07 18:18:21.457880,2022-06-07 18:18:21.475185,66.0,199,None
181,4274,Kayaking on Rhin River,+++++\ninvitation link for trip's whatsapp gro...,48,NaN,no address provided,53.098370,12.897488,True,https://www.meetup.com/Berlin-Outdoor-Adventur...,2022-06-19,09:00:00,2022-06-07 18:27:05.841496,2022-06-07 18:27:05.841496,NaN,199,https://secure-content.meetupstatic.com/images...
576,4662,Bike Trip From Klövensteen Forest to Elbe Rive...,"Hi everybody, who is interested in joining me ...",0,NaN,no address provided,53.580887,9.797163,True,https://www.meetup.com/Hamburg-Hikers/events/2...,2022-06-11,11:35:00,2022-06-07 18:47:54.044534,2022-06-07 18:47:54.044534,NaN,185,https://secure-content.meetupstatic.com/images...


In [54]:
conn = connect_db()
users_df, _, _ = get_data(conn)

/home/daniel/.pyenv/versions/3.8.12/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/daniel/.pyenv/versions/3.8.12/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
translator = Translator()

In [4]:
translator.translate("ich bin gesund").text

'I am healthy'

In [3]:
from_code = "de"
to_code = "en"

# Download and install Argos Translate package
available_packages = argostranslate.package.get_available_packages()

available_package = list(
    filter(
        lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
    )
)[0]
download_path = available_package.download()
argostranslate.package.install_from_path(download_path)

# Translate
installed_languages = argostranslate.translate.get_installed_languages()
from_lang = list(filter(
    lambda x: x.code == from_code,
    installed_languages))[0]
to_lang = list(filter(
    lambda x: x.code == to_code,
    installed_languages))[0]
translation = from_lang.get_translation(to_lang)

In [4]:
def remove_punctuation(text):
    for s in text:
        if s in string.punctuation:
            text = text.replace(s, '')

    return text

In [5]:
def lower_text(text):
    return text.lower()

In [6]:
def remove_numbers(text):
    return ''.join(word for word in text  if not word.isdigit())

In [7]:
def remove_stop_words(text, language='english'):
    lang = detect(text)
    if lang != 'en':
        language = 'german'
    stop_words = set(stopwords.words(language))
    word_tokens = word_tokenize(text)
    
    return ' '.join([w for w in word_tokens if not w in stop_words])

In [8]:
def remove_other_stuff(text):
    # remove http
    text = re.sub(r"http.*\.[a-z]{2,3}","",text)

    # remove www
    text = re.sub(r"www.*\.[a-z]{2,3}","",text)
    
    # remove line breaks
    text = re.sub(r'\n','',text)
    
    return text

In [9]:
def language_translation(string, translator=translation):
    if detect(string) != 'en':
        translatedText = translation.translate(string)
        return translatedText
    else:
        return string

In [10]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()

    return ' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')])

In [11]:
def random_generate_list(x):
    names = list(x.split())
    random.shuffle(names)
    if len(names) > 5:
        names = names[:5]
            
    return names

In [12]:
def random_generate_long_list(x):
    names = list(x.split())
    random.shuffle(names)
    if len(names) > 200:
        names = names[:200]
            
    return names

In [13]:
def create_soup(x):
    soup = (" ".join(x["random_title"]) + " " + " ".join(x["random_description"])).split()
    random.shuffle(soup)
    return ' '.join(soup)

In [14]:
def haversine_vectorized(user_lat,
                         user_lon,
                         event_lat,
                         event_lon):
    """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df
        Computes distance in kms
    """

    lat_1_rad, lon_1_rad = np.radians(user_lat), np.radians(user_lon)
    lat_2_rad, lon_2_rad = np.radians(event_lat), np.radians(event_lon)
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [58]:
%%time
events_df['description_translated'] = events_df.description.apply(language_translation, translator=translation)
events_df['title_translated'] = events_df.title.apply(language_translation, translator=translation)
events_df[['description', 'description_translated']].head()

CPU times: user 42min 20s, sys: 20.6 s, total: 42min 40s
Wall time: 8min 40s


,description,description_translated
0,Berlin's most modern hall for indoor soccer! O...,Berlin's most modern hall for indoor soccer! O...
1,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...","Hello her loved ones,\n\nwe want to play beach..."
2,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...","Hello her loved ones,\n\nwe would like to see ..."
3,no description,no description
4,"Hallo Ihr Lieben,\n\nam Samstag den 25.06.2022...","Hello your loved ones,\n\non Saturday the 25.0..."


In [59]:
events_df['description_punct'] = events_df.description_translated.apply(remove_punctuation)
events_df['title_punct'] = events_df.title_translated.apply(remove_punctuation)
events_df[['description', 'description_punct']].head()

,description,description_punct
0,Berlin's most modern hall for indoor soccer! O...,Berlins most modern hall for indoor soccer Our...
1,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...",Hello her loved ones\n\nwe want to play beach ...
2,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...",Hello her loved ones\n\nwe would like to see o...
3,no description,no description
4,"Hallo Ihr Lieben,\n\nam Samstag den 25.06.2022...",Hello your loved ones\n\non Saturday the 25062...


In [60]:
events_df['description_lower'] = events_df.description_punct.apply(lower_text)
events_df['title_lower'] = events_df.title_punct.apply(lower_text)
events_df[['description', 'description_lower']].head()

,description,description_lower
0,Berlin's most modern hall for indoor soccer! O...,berlins most modern hall for indoor soccer our...
1,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...",hello her loved ones\n\nwe want to play beach ...
2,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...",hello her loved ones\n\nwe would like to see o...
3,no description,no description
4,"Hallo Ihr Lieben,\n\nam Samstag den 25.06.2022...",hello your loved ones\n\non saturday the 25062...


In [61]:
events_df['description_numbers'] = events_df.description_lower.apply(remove_numbers)
events_df['title_numbers'] = events_df.title_lower.apply(remove_numbers)
events_df[['description', 'description_numbers']].head()

,description,description_numbers
0,Berlin's most modern hall for indoor soccer! O...,berlins most modern hall for indoor soccer our...
1,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...",hello her loved ones\n\nwe want to play beach ...
2,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...",hello her loved ones\n\nwe would like to see o...
3,no description,no description
4,"Hallo Ihr Lieben,\n\nam Samstag den 25.06.2022...",hello your loved ones\n\non saturday the we o...


In [62]:
events_df['description_stopwords'] = events_df.description_numbers.apply(remove_stop_words, language='english')
events_df['title_stopwords'] = events_df.title_numbers.apply(remove_stop_words, language='english')
events_df[['description', 'description_stopwords']].head()

,description,description_stopwords
0,Berlin's most modern hall for indoor soccer! O...,berlins modern hall indoor soccer unique blue ...
1,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...",hello loved ones want play beach volleyball we...
2,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...",hello loved ones would like see wednesday th j...
3,no description,no description
4,"Hallo Ihr Lieben,\n\nam Samstag den 25.06.2022...",hello loved ones saturday organize beach volle...


In [63]:
events_df['description_others'] = events_df.description_stopwords.apply(remove_other_stuff)
events_df['title_others'] = events_df.title_stopwords.apply(remove_other_stuff)
events_df[['description', 'description_others']].head()

,description,description_others
0,Berlin's most modern hall for indoor soccer! O...,berlins modern hall indoor soccer unique blue ...
1,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...",hello loved ones want play beach volleyball we...
2,"Hallo ihr Lieben,\n\nwir möchten am Mittwoch d...",hello loved ones would like see wednesday th j...
3,no description,no description
4,"Hallo Ihr Lieben,\n\nam Samstag den 25.06.2022...",hello loved ones saturday organize beach volle...


In [64]:
events_df.to_csv("events_df_savepoint.csv")

In [15]:
events_df = pd.read_csv("events_df_savepoint.csv")

In [ ]:
#events_df['description_lemmatize'] = events_df.description_stopwords.apply(lemmatize_text)
#events_df[['description', 'description_lemmatize']].head()

In [16]:
events_df['random_title'] = events_df['title_others'].apply(lambda x: random_generate_list(x))
events_df['random_description'] = events_df['description_others'].apply(lambda x: random_generate_long_list(x))

In [17]:
events_df['soup'] = events_df.apply(create_soup, axis = 1)

In [18]:
tfidf = TfidfVectorizer(stop_words="english")

In [19]:
tfidf_matrix = tfidf.fit_transform(events_df["soup"])
tfidf_matrix.shape

(904, 7314)

In [20]:
cosine_sim = tfidf_matrix.dot(tfidf_matrix.T)

In [24]:
def get_title(user_id, df):
    ''' this finction grabs the title of an event used by an user from ahoy_events dataset'''
    
    searched_events = []
    for event in df[df['user_id'] == user_id].properties.values:
        searched_events.append(event['offer'])

    searched_events = np.unique(searched_events)

    titles = events_df[events_df['id'].isin(searched_events)].title
    latitude = events_df[events_df['id'].isin(searched_events)].latitude
    longitude = events_df[events_df['id'].isin(searched_events)].longitude
    
    return (titles, latitude, longitude)

In [23]:
events_df.columns

Index(['Unnamed: 0', 'id', 'title', 'description', 'price_per_person',
       'capacity', 'address', 'latitude', 'longitude', 'is_external', 'url',
       'offer_date', 'offer_time', 'created_at', 'updated_at', 'user_id',
       'tag_id', 'external_image_url', 'description_translated',
       'title_translated', 'description_punct', 'title_punct',
       'description_lower', 'title_lower', 'description_numbers',
       'title_numbers', 'description_stopwords', 'title_stopwords',
       'description_others', 'title_others', 'random_title',
       'random_description', 'soup'],
      dtype='object')

In [25]:
print(get_title(66, ahoy_events_df))

(6                        Kayak Tours: Berlin & the World
8                                      Wellenwerk Berlin
180                               Kayaking on Rhin River
192    Spreewald Canoe Tour: Discover the UNESCO bios...
194                            Let's Kayak in SpreeWald 
306                     ⛵️ Segelwochenende Brombachsee⛵️
470    Outdoor Belgium: Canoe trip from Heverlee to L...
475                              Kayaking around Utrecht
480      Fox 22 and Valk Sailing on the Grevelingen Meer
578             Südsee-Camp Wietzendorf/Lüneburger Heide
617                     Drachenboot (Schnupper)-Training
833               Saturday afternoon Sailing on Zürisee 
838                  Full Moon Sailing Zurich H22 Meilen
841                 Zurich sailors social event on land 
Name: title, dtype: object, 6      52.495893
8      52.533422
180    53.098370
192    51.863297
194    52.525743
306    49.129038
470    50.882286
475    52.086443
480    51.673354
578    52.930609
617

In [30]:
def get_user_loc(user_id, df = users_df):   
    ''' this finction grabs the location of an user from ahoy_events dataset'''
    
    users_idx = users_df[users_df['id']== user_id]
    # now we get the user's latitude and longitude
    
    location_dict = {'user_latitude' : float(users_idx.latitude.values), 
                     
                     'user_longitude' : float(users_idx.longitude.values)
                        }
     
    return location_dict

In [38]:
def content_recommender(user_id, cosine_sim, events_df):
    '''This recommender finds out 20 closest matches with a given event'''
    
    #calling user id from ahoy_events
    
    titles, latitude, longitude = get_title(user_id, df = ahoy_events_df)
    
    sim_scores = []
    for idx in events_df[events_df['title'].isin(titles)].index.unique():
        sim_scores.extend(list(enumerate(cosine_sim[idx].todense().tolist()[0])))
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Sort the list of set by sim score, index with highest simlirarity
    # Will be at the beginning of the list
    
    sim_scores = sim_scores[1:101] # top 100 events (first one is our input events so we ignore it)
    
    sport_indices = [i[0] for i in sim_scores] # we grab the indices of those 10 events
    
    df_all_recommendations = events_df.iloc[sport_indices][['id', 'title', 'latitude', 'longitude', 'offer_date']]
    
    #df_date_filter = df_all_recommendations[df_all_recommendations['offer_date'] >= (datetime.datetime.now() + datetime.timedelta(days=3)).date()]
    df_date_filter = df_all_recommendations[df_all_recommendations['offer_date'] >= '2022-06-11']
    
    #calcullation of haversine distance for users:
    user_latitude = get_user_loc(user_id, df = users_df)['user_latitude']
    user_longitude = get_user_loc(user_id, df = users_df)['user_longitude']
    
    df_date_filter['user_latitude'] = user_latitude
    df_date_filter['user_longitude'] = user_longitude
    
    df_date_filter["distance"] = haversine_vectorized(df_date_filter.latitude, df_date_filter.longitude,
                                                      df_date_filter.user_latitude, df_date_filter.user_longitude)
            
    df_location_filter = df_date_filter[df_date_filter['distance'] <= 200.0]
    
    
                                                
    #return np.unique(df_location_filter.id.values)
    return np.unique(df_date_filter.id.values)

In [46]:
(datetime.datetime.now() + datetime.timedelta(days=3)).date()

datetime.date(2022, 6, 11)

In [ ]:
save_npz("model_cosine_sim", cosine_sim)

In [ ]:
matrix_reloaded = load_npz("model_cosine_sim.npz")

In [ ]:
matrix_reloaded.shape, cosine_sim.shape

In [39]:
content_recommender(66, cosine_sim, events_df).tolist()

/tmp/ipykernel_818/1754455029.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_filter['user_latitude'] = user_latitude
/tmp/ipykernel_818/1754455029.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_filter['user_longitude'] = user_longitude
/tmp/ipykernel_818/1754455029.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

[4128,
 4186,
 4187,
 4190,
 4193,
 4194,
 4195,
 4197,
 4198,
 4270,
 4271,
 4272,
 4273,
 4274,
 4275,
 4382,
 4383,
 4547,
 4550,
 4554,
 4555,
 4559,
 4560,
 4561,
 4696,
 4914,
 4915,
 4916,
 4917,
 4918,
 4920,
 4921,
 4922,
 4923,
 4926,
 4928]

In [42]:
events_df[events_df['id'].isin(content_recommender(66, cosine_sim, events_df).tolist())][['title', 'id']]

/tmp/ipykernel_818/1754455029.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_filter['user_latitude'] = user_latitude
/tmp/ipykernel_818/1754455029.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_filter['user_longitude'] = user_longitude
/tmp/ipykernel_818/1754455029.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

,title,id
45,Nature Cycling: Explore one of the most famous...,4128
100,Fairytale Forest: Hike through the mystical la...,4186
101,Fairytale Forest: Hike through the mystical la...,4187
104,Swedish Lake: Join us on an idyllic hike aroun...,4190
107,Peacock’s Paradise: Discover an island full of...,4193
108,Ice Age Reloaded: Discover the glacial lands &...,4194
109,Ice Age Reloaded: Discover the glacial lands &...,4195
111,Mystic River: Venture into haunted forests and...,4197
112,Mystic River: Venture into haunted forests and...,4198
180,Kayaking on Rhin River,4274


In [88]:
result = {}
for user_id in users_df.id.values:
    events = content_recommender(user_id, cosine_sim, events_df).tolist()
    print(events[::-1])
    if user_id not in result.keys():
        result[str(user_id)] = events[::-1]

[]
[]
[]
[]
[]
[]
[4960, 4959, 4958, 4957, 4956, 4953, 4952, 4951, 4950, 4949, 4948, 4947, 4946, 4945, 4944, 4398, 4397, 4389, 4387, 4377, 4376, 4331, 4322, 4321]
[]
[4275, 4274, 4273, 4272, 4271, 4270, 4198, 4197, 4195, 4194, 4193, 4190, 4187, 4186, 4128]
[]


/tmp/ipykernel_2404/1580872386.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_filter['user_latitude'] = user_latitude
/tmp/ipykernel_2404/1580872386.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date_filter['user_longitude'] = user_longitude
/tmp/ipykernel_2404/1580872386.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [89]:
result

{'71': [],
 '72': [],
 '73': [],
 '74': [],
 '68': [],
 '69': [],
 '70': [4960,
  4959,
  4958,
  4957,
  4956,
  4953,
  4952,
  4951,
  4950,
  4949,
  4948,
  4947,
  4946,
  4945,
  4944,
  4398,
  4397,
  4389,
  4387,
  4377,
  4376,
  4331,
  4322,
  4321],
 '67': [],
 '66': [4275,
  4274,
  4273,
  4272,
  4271,
  4270,
  4198,
  4197,
  4195,
  4194,
  4193,
  4190,
  4187,
  4186,
  4128],
 '75': []}

In [90]:
with open('recommender.json', 'w') as fp:
    json.dump(result, fp)

In [82]:
with open('recommender.json', 'r') as fp:
    data = json.load(fp)

In [43]:
upload_file_to_gs('recommender.json')